In [1]:
import torch
from DataFidelities.pytorch_radon.radon import Radon,IRadon
import numpy as np
import pandas as pd
from scipy.io import loadmat
from utils.util import addwgn_torch
from math import sqrt
import pickle
import itertools

In [2]:
CTImageMat=loadmat('CT_images_preprocessed.mat')
CTImageArr=CTImageMat['img_cropped'][:,:,0]
CTImageTensor=torch.tensor(CTImageArr,dtype=torch.float).unsqueeze(0).unsqueeze(0)
theta = torch.tensor(np.linspace(0., 180, 360, endpoint=False),dtype=torch.float)
fp=Radon(in_size=512,theta=theta,circle=False)
sino=fp(CTImageTensor)
noisySino,clampedNoisySino=addwgn_torch(sino,50,0,sqrt(2*512**2))
noisySino=noisySino.detach().clone()
clampedNoisySino=clampedNoisySino.detach().clone()
#with open('noisyView360_clamp.pk', 'wb') as f:
    #pickle.dump(noisySino.cpu(), f)

In [3]:
fullX=torch.tensor(list(itertools.product(range(noisySino.shape[2]),range(noisySino.shape[3])))).float()
fullX[:,0]=fullX[:,0]/noisySino.shape[2]
fullX[:,1]=fullX[:,1]/noisySino.shape[3]
fully=noisySino.reshape((-1,1))
X=fullX.reshape((-1,360,2))[:,::4,:].reshape((-1,2)).detach().clone()
y=noisySino.squeeze()[:,::4].reshape((-1,1)).detach().clone()

In [4]:
Xcsv=pd.DataFrame(X.numpy())
ycsv=pd.DataFrame(y.numpy())
Xcsv.to_csv('X.csv')
ycsv.to_csv('y.csv')

In [5]:
XFcsv=pd.DataFrame(fullX.numpy())
XFcsv.to_csv('fullX.csv')
yFcsv=pd.DataFrame(fully.numpy())
yFcsv.to_csv('fully.csv')
imagecsv=pd.DataFrame(CTImageArr)
imagecsv.to_csv('image.csv')

In [6]:
originFullycsv=pd.DataFrame(sino.reshape(-1,1).detach().cpu().numpy())
originFullycsv.to_csv('originFullyy.csv')

In [7]:
fullX=torch.tensor(list(itertools.product(range(clampedNoisySino.shape[2]),range(clampedNoisySino.shape[3])))).float()
fullX[:,0]=fullX[:,0]/clampedNoisySino.shape[2]
fullX[:,1]=fullX[:,1]/clampedNoisySino.shape[3]
fully=clampedNoisySino.reshape((-1,1))
X=fullX.reshape((-1,360,2))[:,::4,:].reshape((-1,2)).detach().clone()
y=clampedNoisySino.squeeze()[:,::4].reshape((-1,1)).detach().clone()

In [8]:
Xcsv=pd.DataFrame(X.numpy())
ycsv=pd.DataFrame(y.numpy()/sqrt(2*512**2))
Xcsv.to_csv('XClamp.csv')
ycsv.to_csv('yClamp.csv')
XFcsv=pd.DataFrame(fullX.numpy())
XFcsv.to_csv('fullXClamp.csv')
yFcsv=pd.DataFrame(fully.numpy())
yFcsv.to_csv('fullyClamp.csv')